consulta 1: Calcula la distancia promedio y el tiempo promedio de viaje en minutos para todos los viajes realizados en diciembre 2019

In [4]:
import pandas as pd
import os

file_path = os.path.join("..", "data", "processed", "tripdata_clean.csv")
df= pd.read_csv(file_path)

# Calcular el promedio de las columnas
distancia_promedio = df['trip_distance'].mean()
tiempo_promedio_viaje = df['trip_duration_min'].mean()

# Redondear resultados a 2 decimales
distancia_promedio = round(distancia_promedio, 2)
tiempo_promedio_viaje = round(tiempo_promedio_viaje, 2)

# Mostrar resultados
print(f"Distancia promedio en millas: {distancia_promedio}")
print(f"Tiempo promedio de viaje en minutos: {tiempo_promedio_viaje}")

Distancia promedio en millas: 2.98
Tiempo promedio de viaje en minutos: 15.07


Consulta 2: Determina el método de pago más común y cuántos viajes lo usaron. 

In [5]:
import pandas as pd
import os

file_path = os.path.join("..", "data", "processed", "tripdata_clean.csv")
df = pd.read_csv(file_path)

# Diccionario que traduce valores numéricos a nombres de métodos de pago
metodos_pago_diccionario = {
    1: 'tarjeta de crédito',
    2: 'efectivo',
    3: 'sin cargo',
    4: 'en disputa',
    5: 'desconocido',
    6: 'viaje cancelado' 
}

# Mapea los nombres de método de pago en la columna original y luego contar sus ocurrencias.
conteo = df['payment_type'].map(metodos_pago_diccionario).value_counts()

# Obtener método más común y cantidad de viajes que lo usaron
metodo_mas_comun = conteo.idxmax()
cantidad_viajes = conteo.max()

print(f"Método de pago más común: {metodo_mas_comun}")
print(f"Cantidad de viajes que usaron ese método: {cantidad_viajes}")

Método de pago más común: tarjeta de crédito
Cantidad de viajes que usaron ese método: 4736318
